## Tutorial 19: Text analysis with `gensim`

Last week we saw how to use the `XML` module to access and clean the textual
data given in the MediaWiki JSON file. Today, we'll se how to use the `gensim`
module to actually parse the text itself. Eventually you will do all of this 
within the `wikitext.py` module and will not need to call `gensim` directly,
but it will be helpful to understand how it works for some of the later projects.

Start by loading a few functions from the module:

In [ ]:
from gensim import corpora, matutils, models, similarities
from gensim.similarities.docsim import MatrixSimilarity

In this tutorial we will work with a small set of text documents rather than
the longer wikipedia example. This will make it easier to understand exactly
what is going on. Here are the documents we'll use:

In [ ]:
documents = ["Human machine interface for lab human computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

### Tokenization

The first step in most text processing tasks is to split the raw data into individual
words. We have already seen how to do this with regular expressions by detecting sequences
of word characaters. Here we also convert the string to lower case:

In [ ]:
import re
re.findall('(\\w+)', documents[0].lower())

We can create a list of the words in every document by wrapping this up in a `for`
loop:

In [ ]:
word_list = []
for doc in documents:
    word_list.append(re.findall('(\\w+)', doc.lower()))
    
word_list

We have already seen how these tokenized lists, over longer documents, are
useful in detecting the most frequent terms in a given document. Now, we'll
see how to do much more with these words.

### Lexicon

A lexicon, in linguistics proper, refers to all of the terms that a particular
person has at their disposal. In the context of text processing, a lexicon 
refers to all of the terms across of our *corpus* (collection of documents) that
we want to consider. To construct a lexicon using `gensim`, we call the `Dictionary`
object:

In [ ]:
lexicon = corpora.Dictionary(word_list)

One attribue of the lexicon is a mapping from each term into a numeric id. We
can see this with:

In [ ]:
lexicon.token2id

And a particular id comes from using the `token2id` attribute as a dictionary:

In [ ]:
lexicon.token2id['human']

To go in the other direction, treat the lexicon as a list and grab an id
by reference.

In [ ]:
lexicon[3]

### Numeric representation of corpus

Consider the first document in our corpus:

In [ ]:
print(word_list[0])

Using our `lexicon` object it is possible to represent this document a list of integer
ids. We could do this through a complex double `for` loop, but `gensim` provides the
`doc2idx` function to make this easy:

In [ ]:
lexicon.doc2idx(word_list[0])

Why is this numeric representation useful? For one thing, it takes up considerably
less space (at least if we have a large corpus). It is also easier to program with
integer ids. This specific representation is useful, as well, if you want to use
deep learning models for languages. We may have a chance to see these towards the
end of the semester.

### Bag of words

The numeric representation of the terms above does not lose any information in the
original document. A bag of words representation, instead, removes all of the order
information in a document. It simply counts how often each term in the lexicon occurs
within the document. This representation will be essential for nearly all of the methods
we will see for processing text.

The method `doc2bow` of the `lexicon` converts a list of words into a bag of words.
The bag of words is given as a list of tuples given as (word id, count). Here we see,
for example, that the first document uses word number 3 twice:

In [ ]:
lexicon.doc2bow(word_list[0])

We can cycle through the entire set of documents to create the complete bag of words
representation of the corpus.

In [ ]:
bow = []
for t in word_list:
    bow.append(lexicon.doc2bow(t))
    
bow

### Term frequency matrix

A term frequency matrix is an equivalent representation of a bag of words
as a matrix, a rectangular table of numeric values. The table has one row
for each term in the lexicon and one column for each document (I will also
call a matrix with terms in the columns and documents in the rows a term
frequency matrix; context should make it clear which one we are working
with). The method `corpus2dense` produces such as matrix for us:

In [ ]:
tf_array = matutils.corpus2dense(bow, num_terms=len(lexicon.token2id))
tf_array

You will notice that this matrix contains a lot of zero elements. The
preponderance of zeros only gets worse with larger corpora. We can avoid
these by creating instead *sparse* matrix with `corpus2csc`:

In [ ]:
tf_sparse_array = matutils.corpus2csc(bow)
tf_sparse_array

The sparse array can (mostly) be manipulated just like the dense version, but saves
a lot of space and (sometimes) is much faster to operate on. We won't need either of
these term frequency matricies directly until we start building predictive models
in a few weeks, but they are a nice way of thinking about the bag of words model.

### Term frequency inverse document frequency

If we take the term frequency matrix and determine the largest values in each column,
we will get the most frequent terms within each document. We've seen that this can be
somewhat useful for understanding the content of a Wikipedia page. However, we have 
also seen that many of the terms are not very interesting but are instead commonly
used across the entire corpus (either grammatic words like "the", "and", "because"
or words that would be appear in anything related to our topic such as "cake" or "Virginia").

A solution is to also weight each term as a function of how often it occurs in all
documents. There are several ways to do this; I'll explain just the most common
technique. Let $f_{t, d}$ count the number of times that term $t$ occurs in document
$d$, $n_t$ the number of documents that use term $t$ at least once, and $D$ be the
total number of documents. Then, for each term $t$ and document $d$ we have the 
tf-idf (term frequency inverse document frequency) of:

$$ tfidf(t, d) = f_{t, d} \times log_2 \left( \frac{D}{n_t} \right) $$

So, the score will be higher if the term is used more frequently in a document but
lower if the term is used in more documents. **The idea is that terms with the highest
tfidf score for a given document are the most distinguishing ones for that particular
document.**

We can build a tfidf model using the `TfidfModel` method:

In [ ]:
tfidf = models.TfidfModel(bow)

The model can then be applied to any particular document of interest:

In [ ]:
tfidf[bow[1]]

The most relevant term in document 1 is then term number 9, even
though it is used only once:

In [ ]:
lexicon[9]

We can visually determin the most interesting term for a single document,
but it is useful to automatically sort the list for use in general programming.

In [ ]:
tf_obj = tfidf[bow[1]]
sorted(tf_obj, key=lambda x: x[1], reverse=True)[:5]

And now we see the top five terms for this particular document:

In [ ]:
n_terms = 5

top_terms = []
for obj in sorted(tf_obj, key=lambda x: x[1], reverse=True)[:n_terms]:
    top_terms.append("{0:s} ({1:01.03f})".format(lexicon[obj[0]], obj[1]))

print(top_terms)

It is possible to also represent the TF-IDF object as a matrix, as show by
the following code:

In [ ]:
tfidf_corpus = []
for doc in bow:
    tfidf_corpus.append(tfidf[doc])

tfidf_mat = matutils.corpus2dense(tfidf_corpus, num_terms=len(lexicon.token2id))
tfidf_mat[:40, :4]

Now the values are weighted according to how frequent the word is across the
corpus.

### Similarity

Consider each column of the term frequency of TF-IDF matrix. This sequence
of numbers describes the terms used with a given document. In other words,
we have projected each document in a D-dimensional space (where $D$ is the
number of terms in our lexicon). Now that our documents are just points in
space, there are a lot *mathy* things we can do with them. One example is
determining which documents are similar to which other documents. We could
just compute the (Euclidean) distance between two documents, but there is
another metric that is popular in text analysis called *cosine similarity*.
It measures the angle between two documents; the details are not important
but what matters is the cosine similarity is not sensitive to how long a
particular document is.

We can apply cosine similarty to either the TF matrix or the TF-IDF matrix.
The latter is generally recommended. Here, we use the `MatrixSimilarity` 
class to compute a similarity score for our corpus.

In [ ]:
matsim = MatrixSimilarity(tfidf_corpus, num_features=len(lexicon))

As with the TF-IDF class, we need to apply this similarity score to a particular document.
Here we apply it to document 1:

In [ ]:
matsim[tfidf_corpus[1]]

A cosine similarity score of 1 means that two documents are perfectly identical; notice
that document 1 has a score of $0.99999994$ to itself due to rounding errors. The next
most similar document is document 8, probably because both use the rare term "survey":

In [ ]:
print("Document 1: '" + documents[1] + "'")
print("Document 8: '" + documents[8] + "'")

As with the term frequency measurement, we could automate detecting the closest document
and use this to explore a corpus of text. And again, we could think of these similarities
as a matrix, here a square one with one row and column for each document:

In [ ]:
import numpy as np
np.round(matsim[tfidf_corpus], 3)

Notice that, when rounded, the diagonal elements are all one. Why is this?

### Clustering

Another task we can do given that our documents are projected in a high-dimensional
numeric space is to cluster the documents according to the words they use. We have
already seen how to do this with the network data; we are just doing this same thing
now with the words themselves.

Due to something called the *curse of dimensionality*, applying classical clustering
techniques to our data will not work very well. There are too many dimensions, the 
number of words in our lexicon, to reasonably do any clustering of the documents.
One clustering technique that does work very well works directly with a matrix of
similarity scores; it is called *spectral clustering* and we can apply it as follows:

In [ ]:
from sklearn.cluster import SpectralClustering


scmodel = SpectralClustering(n_clusters=3, affinity='precomputed')
similarity_matrix = matsim[tfidf_corpus]
scmodel.fit_predict(similarity_matrix)

The spectral clustering splits our documents into three groups and returns the id
of each group. If you want to know more about the clustering algorithm itself, I'll
put some notes on the main course site. It's hard to explain the technique unless 
you've had linear algebra.

### Topic Models

A topic model is a probabilistic technique for understanding *topics* that occur
within a corpus. Here, topics are understood as groups of words that tend to co-occur
within documents. For example, the words 'flour', 'oil', 'sugar', and 'oven' may
all group together to form a topic about baking. By far the most popular technique
for detecting topics is an approach called Latent Dirchlet Allocation, or LDA.
Here we will use `gensim` to apply the model to our corpus:

In [ ]:
from gensim.models import LdaModel
lda = LdaModel(bow, id2word=lexicon, num_topics=3, alpha='auto', iterations=50)

Here are the most highly weighted words for each of the three topics (this corpus
is far too small for the output to really make much sense):

In [ ]:
lda.show_topics(num_words=3)

Likewise, we can see how much of each document is associated with each topic:

In [ ]:
list(lda.get_document_topics(bow))

I have also added a reference on the course website about LDA that does not require
understanding all of the underlying probability theory.

### Stopwords and lexicon creation

Our final topic for this tutorial is how to manually reduce the number of 
terms in our lexicon. There are two reasons we may want to remove a term
from the lexicon:

1. Terms that occur too frequently, or are *function* words that serve a
primarly grammatical function, may result in erroneous results particularly
in our topic models.
2. There will be a large number of occurs that only occur a very small number
of documents. These are generally not very interesting (mispelled words or
proper names) and can be removed to save time and space. They also may cause
issues in TF-IDF as they become too heavily weighted.

There is an easy method `filter_extremes` that removes terms based on how 
frequently they are used in the corpus. Here we keep only those terms that
are used in at least two documents and in no more than 70% of the documents.

In [ ]:
print(len(lexicon))
lexicon.filter_extremes(no_below=2, no_above=0.7)
print(len(lexicon))

The lexicon now has only 16 terms from the original 41. We can also use a list
of pre-defined terms that we want to remove, known as stopwords. Here is a common
list of English terms that we will make use of this semester:

In [ ]:
with open('ranksnl_large.txt', 'r') as fin:
    sw_list = fin.read().splitlines()
    
print(sw_list)

The following code removes these terms from our lexicon.

In [ ]:
sw_list = set(sw_list).intersection(lexicon.token2id.keys())
ids = [lexicon.token2id[x] for x in sw_list]
lexicon.filter_tokens(ids)
len(lexicon)

The result now contains four fewer terms. 